In [43]:
# base imports
import pandas as pd
import numpy as np

In [16]:
# Column met_o had values 1/2. It needs to be changed to 0/1.
df = pd.read_csv('./data_for_matrix.csv')
df['met_o'] = df['met_o'].apply(lambda x: x - 1)
df.to_csv('data_for_matrix.csv', index=False)

### Below you can see code that prepares data for the basic matrix factorization.
Here in the base matrix we only have information about match. So we need a set of vectors, where each vector describes each date (holds both ids and information about match).

Task will requite two base matrices.
1. Matrix where men are "users" and women are "products". It will then be used to recommend women to men because matrix will say what's the predicted rating of a woman in eyes of man. Basically it will answer the question: **"How likely is that a man will like a woman?"**. Let's call this matrix/data frame **"men_like_women"**.
1. Matrix where women are "users" and men are "products". It will then be used to recommend men to women because matrix will say what's the predicted rating of a man in eyes of woman. Basically it will answer the question: **"How likely is that a woman will like a man?"**. Let's call this matrix/data frame **"women_like_men"**.

Why such analogies? It may help to understand how do this human relations task translates into recommender systems world.

These matrices will be used to train models (with matrix factorization) which will then be saved into csv files.

In [54]:
# Split into vectors, let's have two matrices as describes above.
base_df = pd.read_csv('./data_for_matrix.csv')
men_like_women_data = []
women_like_men_data = []

for _, row in base_df.iterrows():
    vector = {
        'id': int(row['iid']),
        'pid': int(row['pid']),
        'match': int(row['match'])
    }
    if row['gender'] == 0:
        women_like_men_data.append(vector)
    else:
        men_like_women_data.append(vector)

men_like_women_df = pd.DataFrame(men_like_women_data)
women_like_men_df = pd.DataFrame(women_like_men_data)

print(men_like_women_df)
print(women_like_men_df)

       id  pid  match
0      11    1      0
1      11    2      0
2      11    3      0
3      11    4      0
4      11    5      0
...   ...  ...    ...
4179  552  526      0
4180  552  527      0
4181  552  528      0
4182  552  529      0
4183  552  530      0

[4184 rows x 3 columns]
       id  pid  match
0       1   11      0
1       1   12      0
2       1   13      1
3       1   14      1
4       1   15      1
...   ...  ...    ...
4179  530  548      0
4180  530  549      0
4181  530  550      0
4182  530  551      0
4183  530  552      0

[4184 rows x 3 columns]
